In [1]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader
from langchain_community.llms import GPT4All
import llama_index

C:\Users\sarth\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
reader = SimpleDirectoryReader(
    input_files=["goog-10-k-2023.pdf","tsla-20231231-gen.pdf","uber-10-k-2023.pdf"]
)
documents = reader.load_data()

print('Number of pages:', len(documents))
print(documents)

Number of pages: 381
[Document(id_='b081bb96-1003-467d-99fc-f92716798e4c', embedding=None, metadata={'page_label': '1', 'file_name': 'goog-10-k-2023.pdf', 'file_path': 'goog-10-k-2023.pdf', 'file_type': 'application/pdf', 'file_size': 926655, 'creation_date': '1980-01-01', 'last_modified_date': '2024-07-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n___________________________________________\nFORM 10-K  \n___________________________________________\n(Mark One)\n☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended December 31, 2023  \nOR\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHAN

In [3]:
parser = llama_index.core.node_parser.SentenceSplitter.from_defaults(chunk_size=512, chunk_overlap=20)
nodes = parser.get_nodes_from_documents(documents)

In [13]:
llm = GPT4All(model="./qwen2-0_5b-instruct-q8_0")

In [5]:
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="./sentence-transformers")
)

d:\Softwares\Python\anaconda3\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
## build a new index
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.deeplake import DeepLakeVectorStore


# construct vector store and customize storage context
dl_vector_store = DeepLakeVectorStore(dataset_path="dataset")
storage_context = StorageContext.from_defaults(vector_store=dl_vector_store)
# Load documents and build index
index = VectorStoreIndex(
    nodes, storage_context=storage_context,embed_model=embed_model
)

Deep Lake Dataset in dataset already exists, loading from the storage
Uploading data to deeplake dataset.


100%|██████████| 951/951 [00:02<00:00, 454.48it/s]


Dataset(path='dataset', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (951, 768)  float32   None   
    id        text      (951, 1)     str     None   
 metadata     json      (951, 1)     str     None   
   text       text      (951, 1)     str     None   


In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.deeplake import DeepLakeVectorStore
vector_store = DeepLakeVectorStore(dataset_path="suni-dataset",read_only=True,)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store,embed_model=embed_model)

In [8]:
qe = index.as_query_engine(llm=llm)

In [11]:
response = qe.query("What is the total revenue for Google Search")

In [12]:
print(response.response)

282,836

The document does not provide a specific value for the total revenue of Google Search & other. The text states that revenues are presented by type (in millions) in year ended December 31, 2022 and then again in years ending December 31, 2023.

Therefore, to find the total revenue for Google Search & other, we need to look at the data from those two years. The text states that revenues are presented by type (in millions) in year ended December 31, 2022 and then again in years ending December 31, 2023.

The document does not provide a specific value for Google Search & other revenue for the period from 2022 to 2023. The text states that revenues are presented by type (in millions) in year ended December 31, 2022 and then again in years ending December 31, 2023.

So, we cannot determine the total revenue for Google Search & other from this document without prior knowledge or context. The text does not provide a specific value for that amount. 

The correct answer is: Total revenu